In [10]:
import cv2
from mtcnn import MTCNN
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import numpy as np

# Load MTCNN model for face detection
detector = MTCNN()

# Load FaceNet model for face embedding extraction
facenet_model = tf.keras.applications.ResNet50(weights='imagenet')  # Provide the path to your FaceNet model

def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values
    return img

def extract_face_embeddings(frame):
    faces = detector.detect_faces(frame)
    face_embeddings = []
    for face in faces:
        x, y, w, h = face['box']
        x1, y1 = max(x, 0), max(y, 0)
        x2, y2 = min(x + w, frame.shape[1]), min(y + h, frame.shape[0])
        cropped_face = frame[y1:y2, x1:x2]
        
        # Preprocess the face for FaceNet model
        preprocessed_face = preprocess_image(cropped_face)
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Get face embeddings using FaceNet model
        face_embedding = facenet_model.predict(preprocessed_face)
        face_embeddings.append(face_embedding)
    return face_embeddings

# Read video
video_capture = cv2.VideoCapture('MELD.Raw\dev_splits_complete\dia1_utt0.mp4')  # Provide the path to your input video

all_embeddings = []  # List to store embeddings for all frames

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    
    # Extract face embeddings from each frame
    extracted_embeddings = extract_face_embeddings(frame)
    
    all_embeddings.append(extracted_embeddings)  # Append embeddings for this frame
    
    # Display the frame with bounding boxes around detected faces
    for face in detector.detect_faces(frame):
        x, y, w, h = face['box']
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

# 'all_embeddings' now contains embeddings for all frames


1/1 [==============================] - 0s 36ms/step


In [13]:
for i in range(len(all_embeddings)):
    if len(all_embeddings[i]) > 1:
        print(i)
print('done')

done
